In [1]:
from google.colab import files

files.upload()

!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d dsluciano/daigt-one-place-all-data

Saving kaggle.json to kaggle.json
 73% 25.0M/34.2M [00:00<00:00, 48.3MB/s]
100% 34.2M/34.2M [00:00<00:00, 64.4MB/s]


In [ ]:
!unzip daigt-one-place-all-data.zip


Archive:  daigt-one-place-all-data.zip
  inflating: concatenated.csv        


In [ ]:
import pandas as pd

data = pd.read_csv('concatenated.csv')

data.head(2)

,id,prompt_id,text,generated,model,kaggle_repo
0,d429f032,0,Advantages of Limiting Car Usage \n\nLimiting ...,1,gpt-3.5-turbo,1
1,1ce279be,0,Advantages of Limiting Car Usage\n\nLimiting c...,1,gpt-3.5-turbo,1


In [ ]:
data.shape

(54691, 6)

In [ ]:
data = data.drop(['id'], axis=1)
data = data.drop(['prompt_id'], axis = 1)

In [ ]:
data

,text,generated,model,kaggle_repo
0,Advantages of Limiting Car Usage \n\nLimiting ...,1,gpt-3.5-turbo,1
1,Advantages of Limiting Car Usage\n\nLimiting c...,1,gpt-3.5-turbo,1
2,Limiting car usage has numerous advantages tha...,1,gpt-3.5-turbo,1
3,The passages provided discuss the advantages o...,1,gpt-3.5-turbo,1
4,Title: The Advantages of Limiting Car Usage\n\...,1,gpt-3.5-turbo,1
...,...,...,...,...
54686,I believe using cellphones in class for educat...,0,human,9
54687,"Working alone, students do not have to argue w...",0,human,9
54688,"""A problem is a chance for you to do your best...",0,human,9
54689,Many people disagree with Albert Schweitzer's ...,0,human,9


In [ ]:
data.model.unique()

data

array(['gpt-3.5-turbo', 'gpt-4', 'unknown', 'falcon-180b', 'llama-70b',
       'llama-falcon', 'palm-2', 'claude', 'human', 'mistral'],
      dtype=object)

In [ ]:
data = data.sample(frac =1)

In [ ]:
data.head(4)

,text,generated,model,kaggle_repo
17522,Driverless cars have become a hot topic of deb...,1,llama-falcon,4
16327,"""Making Mona Lisa Smile,"" an article that disc...",1,llama-falcon,4
27390,Extracurricular activities can give you many b...,0,human,8
31452,Do you ever sit in class so confused that you ...,0,human,8


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data['text'].to_list(), data['generated'].to_list(), test_size = 0.2, random_state=42)

In [ ]:
len(x_train), len(x_test)

(43752, 10939)

In [ ]:
x_train

In [ ]:
import tensorflow as tf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create baseline model
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), #coverts words to  numbers
    ('clf', MultinomialNB()),
])


model_0.fit(
    x_train, y_train
)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
model_0.score( x_test,
              y_test)

0.9527379102294542

In [ ]:
model_0.predict([x_train[0]])

array([0])

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_preds):
  """
    Calculate Model accuracy, precision, recall and f1Score  of a binary classification model

    Args:
      y_true
      y_preds

  """
  #Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_preds) * 100
  #Calculate model precison, recall and f1score using 'weighted' average
  model_precision, model_recall, model_f1score, _ = precision_recall_fscore_support(y_true, y_preds, average = 'weighted')

  model_results = {'accuracy': model_accuracy,
                   "precision": model_precision,
                   "recal": model_recall,
                   "f1": model_f1score}

  return model_results

In [ ]:
model0_preds = model_0.predict(x_test)

baseline_results = calculate_results(y_test, model0_preds)

In [ ]:
baseline_results

{'accuracy': 95.27379102294542,
 'precision': 0.954589496854882,
 'recal': 0.9527379102294542,
 'f1': 0.9525431607503754}

In [ ]:
import pickle
with open('model.pkl','wb') as f:
    pickle.dump(model_0,f)
